<a href="https://colab.research.google.com/github/Robsonzk/aula_dio_agentes_sq/blob/main/C%C3%B3pia_de_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY_1')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)


# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. A melhor forma de saber quando a próxima edição será realizada é:

*   **Acompanhar as redes sociais da Alura:** Fique de olho no Instagram, LinkedIn, Twitter e Facebook da Alura, pois eles costumam anunciar novos eventos e imersões por lá.
*   **Assinar a newsletter da Alura:** Ao se inscrever na newsletter, você receberá informações sobre os próximos cursos, imersões e eventos.
*   **Visitar a página de Imersões da Alura:** Verifique periodicamente a página de Imersões no site da Alura para ver se há alguma edição futura da Imersão IA com Google Gemini programada.

Você pode encontrar os links para as redes sociais e newsletter no site oficial da Alura: [https://www.alura.com.br/](https://www.alura.com.br/)

In [ ]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    # Inserir a tool de busca do Google ###############################################
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura já aconteceu entre os dias 12 e 16 de maio de 2025. As inscrições se encerraram no dia 11 de maio de 2025.

In [ ]:
# Exibe a busca
print(f"Busca realizada: {resposta.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in resposta.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(resposta.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['próxima imersão IA com Google Gemini Alura']
Páginas utilizadas na resposta: starten.tech, alura.com.br



In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk


In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################

def agente_buscador(topico, data_de_hoje):
  buscador = Agent(
      name="agente_buscador_de_noticias",
      model="gemini-2.0-flash",
      instruction="""
      Você é um agente que busca notícias sobre tecnologia no Google. A sua terefa é usar a ferramenta de pesquisa do (google_search)
      para recuperar as últimas noticias de laçamentos relevantes sobre tecnologia.
      Foque em no máximo 10 noticiais relevantes, com base na quantidade de entusiasmo relevantes sobre elas.
      Se o tema tiver poucas notícias ou reações intusiasmada, é possivel que ela não seja tão relevante assim como você acha
      e pode ser subistituida por outra que seja mais relevante.
      Esses lançamentos devem ser atuais de no máximo 30 dias antes da data de hoje.
      """,
      description="Agente que busca notícias no google sobre o tecnologia",
      tools=[google_search]

  )

  entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
  lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)

  return lancamentos_buscados
  # Executa o agente

In [ ]:
################################################
# --- Agente 2: Planejador de posts de tecnologias para sites --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de post, especialista de sites de tecnologia. Com base na lista de lançamentos mais recentes e relevantes buscados, você deve: usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderiamos abordar em um post sobre cada um deles. Você também pode usar o (google_serch) para encontrar mais
        informações sobre o tema e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre elas com base em suas pesquisas
        e retonar esse tema, seus pontos mais relevantes, e um plano com os assuntos a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts de tecnologia para sites",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [ ]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator_de_site_de_tecnologia",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para sites.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Site sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Sites"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [ ]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para Sites, com foco em tecnologia.
            Por ter um público jovem, entre 0 e 90 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para sites."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Sites com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências de tecnologia: ")

# Inserir lógica do sistema de agentes ################################################

if not topico:
  print("Você esqueceu de digitar o tópico!")
else:
  print(f"Maravilha! Vamos criar o post sobre novidades em {topico}")

  lancamentos_buscados = agente_buscador(topico, data_de_hoje)
  print("\n--- Resultado Agente 1  (Buscador)  ---\n")
  display(to_markdown(lancamentos_buscados))
  print("------------------------------------------------------------")

  plano_de_post = agente_planejador(topico, lancamentos_buscados)
  print("\n--- Resultado Agente 2  (Planejador)  ---\n")
  display(to_markdown(text=plano_de_post))
  print("------------------------------------------------------------")

  rascunho_de_post = agente_redator(topico, plano_de_post)
  print("\n--- Resultado Agente 3  (Redator)  ---\n")
  display(to_markdown(text=rascunho_de_post))
  print("------------------------------------------------------------")

  post_final = agente_revisor(topico, rascunho_de_post)
  print("\n--- Resultado Agente 4  (Revisor)  ---\n")
  display(to_markdown(text=post_final))
  print("------------------------------------------------------------")


🚀 Iniciando o Sistema de Criação de Posts para Sites com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências de tecnologia: agentes e IA
Maravilha! Vamos criar o post sobre novidades em agentes e IA

--- Resultado Agente 1  (Buscador)  ---



> Para encontrar as notícias mais relevantes sobre agentes e IA nos últimos 30 dias, vou realizar algumas pesquisas no Google.
> 
> 
> Com base nas notícias encontradas, aqui estão alguns dos lançamentos e tendências mais relevantes sobre agentes e IA nos últimos 30 dias:
> 
> 1.  **Crescimento e Adoção de Agentes de IA:**
> 
> *   O mercado global de agentes de IA deverá atingir US$ 52,62 bilhões até 2030. (Fonte: Mercado de agentes de IA deverá crescer para 52,6 mil milhões de dólares até 2030)
> *   25% das empresas adotarão agentes inteligentes em 2025, com projeções de aumento para 50% até 2027 e 80% até 2026. (Fonte: Avanço da IA: 25% das empresas adotarão agentes inteligentes em 2025)
> *   Empresas estão investindo fortemente no aprimoramento desses sistemas, tornando-os mais personalizados e eficientes. (Fonte: 2025: o ano que os agentes de inteligência artificial saem dos bastidores)
> *   A crescente necessidade de automação eficiente e decisões mais rápidas está impulsionando a adoção de agentes de IA. (Fonte: Mercado de agentes de IA deverá crescer para 52,6 mil milhões de dólares até 2030)
> 
> 2.  **Aplicações e Impacto nas Empresas:**
> 
> *   Agentes de IA estão sendo implementados em setores como e-commerce, atendimento ao cliente, marketing e finanças. (Fonte: 2025: o ano que os agentes de inteligência artificial saem dos bastidores)
> *   Esses agentes não apenas respondem a perguntas, mas também analisam dados e sugerem soluções em tempo real. (Fonte: 2025: o ano que os agentes de inteligência artificial saem dos bastidores)
> *   A adoção de agentes de IA transforma a gestão de processos, simplificando tarefas e otimizando recursos. (Fonte: Adoção de Agentes de IA nas Empresas Transforma Gestão de Processos em 2025)
> *   Empresas que implementaram agentes de IA relatam melhorias significativas nas operações, com funcionários experimentando um aumento de mais de 60% na eficiência. (Fonte: 10 melhores agentes de IA para automação de negócios (2025))
> 
> 3.  **Tecnologias e Inovações:**
> 
> *   A IBM está inovando com agentes especializados e expandindo as capacidades de integração e customização do watsonx Orchestrate. (Fonte: Adoção de Agentes de IA nas Empresas Transforma Gestão de Processos em 2025)
> *   A OpenAI está se destacando no mercado corporativo com um aumento significativo nas assinaturas de seus modelos e ferramentas de IA. (Fonte: Adoção de Agentes de IA nas Empresas Transforma Gestão de Processos em 2025)
> *   A funcionalidade de agentes personalizados lançada pela OpenAI permite que organizações treinem soluções de IA para atender demandas específicas. (Fonte: Agentes de Inteligência Artificial: Transformação 2025)
> 
> 4.  **Desafios e Reflexões:**
> 
> *   Há debates sobre o que realmente define um "agente de IA", com alguns especialistas argumentando que eles não são muito mais do que LLMs com capacidade de raciocínio e execução em múltiplos passos. (Fonte: Ningúém sabe o que são agentes de IA, dizem sócios da a16z)
> *   Especialistas enfatizam que os agentes de IA de hoje podem tomar decisões e agir, mas ainda não são capazes de substituir completamente um humano em muitos trabalhos que exigem criatividade e pensamento. (Fonte: Ningúém sabe o que são agentes de IA, dizem sócios da a16z)
> 
> 5.  **O Futuro dos Agentes de IA:**
> 
> *   A IA generativa está evoluindo para agentes inteligentes autônomos com capacidade de tomar decisões. (Fonte: Futuro da IA: Agentes Inteligentes com Arquiteturas Orientadas a Eventos)
> *   A Arquitetura Orientada a Eventos (EDA) emerge como uma solução essencial para criar sistemas de IA escaláveis, permitindo que agentes operem como microserviços inteligentes que se comunicam assincronamente. (Fonte: Futuro da IA: Agentes Inteligentes com Arquiteturas Orientadas a Eventos)
> 
> Essas notícias indicam que os agentes de IA estão se tornando cada vez mais importantes no cenário tecnológico e empresarial, com um potencial significativo para transformar a forma como as empresas operam e interagem com seus clientes.
> 


------------------------------------------------------------

--- Resultado Agente 2  (Planejador)  ---



> Com base nos lançamentos e tendências que você coletou sobre agentes de IA, podemos elaborar um plano de conteúdo focado no impacto e nas aplicações práticas dessa tecnologia em ascensão.
> 
> Para aprofundar a pesquisa, podemos usar as seguintes pesquisas no Google:
> 
> 
> ## Tema Mais Relevante: Adoção e Impacto dos Agentes de IA nas Empresas
> 
> **Justificativa:** Este tema é o mais abrangente e de maior interesse para o público empresarial. Ele permite explorar tanto o crescimento do mercado quanto os benefícios práticos e desafios da implementação de agentes de IA.
> 
> **Pontos Mais Relevantes:**
> 
> *   **Crescimento e Adoção:**
>     *   Aumento significativo do mercado global de agentes de IA.
>     *   Projeções de adoção em larga escala pelas empresas nos próximos anos.
>     *   Investimentos contínuos para aprimorar a personalização e eficiência dos agentes.
> *   **Aplicações e Impacto:**
>     *   Exemplos de uso em diversos setores: e-commerce, atendimento ao cliente, finanças, etc.
>     *   Agentes de IA como ferramentas de análise de dados e tomada de decisão em tempo real.
>     *   Transformação na gestão de processos, otimização de recursos e aumento da eficiência.
>     *   Melhorias significativas nas operações e aumento da eficiência dos funcionários.
> *   **Desafios e Reflexões:**
>     *   Debates sobre a definição exata de "agente de IA".
>     *   Limitações atuais dos agentes de IA em relação à criatividade e pensamento humano.
> 
> ## Plano de Conteúdo Detalhado:
> 
> **Título:** Agentes de IA: Como estão Transformando as Empresas em 2025
> 
> **Objetivo:** Informar o público sobre o crescimento, aplicações, impacto e desafios dos agentes de IA no ambiente empresarial atual.
> 
> **Público-Alvo:** Gestores, líderes de TI, tomadores de decisão e profissionais interessados em inovação e automação de processos.
> 
> **Estrutura do Post:**
> 
> 1.  **Introdução:**
>     *   Contextualizar o rápido avanço da IA e o surgimento dos agentes de IA.
>     *   Apresentar o tema central e a relevância dos agentes de IA para as empresas.
>     *   Destacar o potencial transformador dessa tecnologia.
> 2.  **O Crescimento Exponencial dos Agentes de IA:**
>     *   Estatísticas sobre o crescimento do mercado global de agentes de IA.
>     *   Projeções de adoção em larga escala pelas empresas nos próximos anos (2025-2027).
>     *   Fatores que impulsionam esse crescimento: automação, eficiência, decisões rápidas.
> 3.  **Aplicações Práticas dos Agentes de IA nas Empresas:**
>     *   Setores que estão liderando a adoção: e-commerce, atendimento ao cliente, marketing, finanças.
>     *   Exemplos de uso em diferentes áreas:
>         *   Atendimento ao cliente: chatbots avançados, suporte técnico automatizado.
>         *   Marketing: personalização de campanhas, análise de dados de clientes.
>         *   Finanças: análise de risco, detecção de fraudes, otimização de investimentos.
>         *   E-commerce: recomendação de produtos, otimização de preços, gerenciamento de estoque.
>     *   Como os agentes de IA estão simplificando tarefas e otimizando recursos.
> 4.  **Impacto nos Resultados e na Eficiência Operacional:**
>     *   Melhorias significativas nas operações e aumento da eficiência dos funcionários.
>     *   Dados sobre o aumento da eficiência em empresas que implementaram agentes de IA.
>     *   Como os agentes de IA estão transformando a gestão de processos.
>     *   Estudos de caso de empresas que obtiveram sucesso com a implementação de agentes de IA.
> 5.  **Desafios e Reflexões:**
>     *   Debates sobre a definição exata de "agente de IA".
>     *   Argumentos de especialistas sobre as capacidades e limitações dos agentes de IA.
>     *   Discussão sobre o papel dos agentes de IA em relação ao trabalho humano.
>     *   Considerações éticas e de segurança ao implementar agentes de IA.
> 6.  **O Futuro dos Agentes de IA:**
>     *   Tendências e inovações em desenvolvimento: IA generativa, agentes autônomos.
>     *   Arquitetura Orientada a Eventos (EDA) como solução para sistemas de IA escaláveis.
>     *   Previsões sobre o impacto dos agentes de IA nos próximos anos.
> 7.  **Conclusão:**
>     *   Recapitular os principais pontos abordados no post.
>     *   Reforçar a importância dos agentes de IA para as empresas em 2025.
>     *   Incentivar os leitores a explorar e implementar essa tecnologia em seus negócios.
>     *   Deixar uma mensagem final sobre o futuro promissor dos agentes de IA.
> 
> Este plano detalhado fornecerá uma base sólida para criar um post informativo e envolvente sobre o impacto dos agentes de IA nas empresas em 2025.
> 


------------------------------------------------------------

--- Resultado Agente 3  (Redator)  ---



> ## Agentes de IA: Como estão Transformando as Empresas em 2025
> 
> A Inteligência Artificial (IA) não para de evoluir, e uma das áreas que mais tem chamado a atenção é a dos **agentes de IA**. Mas o que são exatamente e como estão impactando o mundo dos negócios? Prepare-se para descobrir como essa tecnologia está remodelando o futuro das empresas!
> 
> ### O Crescimento Exponencial dos Agentes de IA
> 
> O mercado global de agentes de IA está em franca expansão, com projeções de adoção em larga escala pelas empresas nos próximos anos. Esse crescimento é impulsionado pela necessidade de **automatização**, **eficiência** e **tomada de decisões rápidas**. As empresas estão investindo cada vez mais em soluções que utilizam agentes de IA para otimizar processos e melhorar seus resultados.
> 
> ### Aplicações Práticas dos Agentes de IA nas Empresas
> 
> Os agentes de IA estão sendo aplicados em diversos setores, como **e-commerce**, **atendimento ao cliente**, **marketing** e **finanças**. Veja alguns exemplos práticos:
> 
> *   **Atendimento ao Cliente:** Chatbots avançados que oferecem suporte técnico automatizado e resolvem dúvidas dos clientes de forma eficiente.
> *   **Marketing:** Personalização de campanhas com base na análise de dados dos clientes, aumentando a taxa de conversão e a fidelização.
> *   **Finanças:** Análise de risco, detecção de fraudes e otimização de investimentos, garantindo a segurança e a rentabilidade das operações.
> *   **E-commerce:** Recomendação de produtos, otimização de preços e gerenciamento de estoque, melhorando a experiência do cliente e aumentando as vendas.
> 
> Esses agentes estão simplificando tarefas, otimizando recursos e liberando os colaboradores para atividades mais estratégicas.
> 
> ### Impacto nos Resultados e na Eficiência Operacional
> 
> A implementação de agentes de IA tem gerado melhorias significativas nas operações e aumentado a eficiência dos funcionários. Empresas que adotaram essa tecnologia relatam um aumento expressivo na produtividade e uma redução de custos operacionais. A gestão de processos também tem sido transformada, tornando-se mais ágil e eficiente.
> 
> ### Desafios e Reflexões
> 
> Apesar de todo o potencial, ainda existem debates sobre a definição exata de "agente de IA" e discussões sobre suas capacidades e limitações. É importante lembrar que, embora os agentes de IA sejam capazes de realizar tarefas complexas, eles ainda não possuem a criatividade e o pensamento crítico dos seres humanos.
> 
> Além disso, é fundamental considerar as questões éticas e de segurança ao implementar agentes de IA, garantindo que sejam utilizados de forma responsável e transparente.
> 
> ### O Futuro dos Agentes de IA
> 
> O futuro dos agentes de IA é promissor, com tendências e inovações em desenvolvimento, como a IA generativa e os agentes autônomos. A Arquitetura Orientada a Eventos (EDA) surge como uma solução para criar sistemas de IA escaláveis e adaptáveis. Nos próximos anos, os agentes de IA devem se tornar ainda mais inteligentes e integrados ao nosso dia a dia.
> 
> ### Conclusão
> 
> Os agentes de IA estão transformando as empresas em 2025, impulsionando a inovação e a eficiência. Se você busca se destacar no mercado, é fundamental explorar e implementar essa tecnologia em seus negócios. O futuro é agora, e os agentes de IA são a chave para o sucesso!
> 
> #InteligenciaArtificial #AgentesDeIA #Inovacao #Tecnologia
> 


------------------------------------------------------------

--- Resultado Agente 4  (Revisor)  ---



> O rascunho está ótimo e pronto para publicar!


------------------------------------------------------------
